In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize
import re
from gensim.models import doc2vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
import keras
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import LSTM, Dense, Bidirectional, Input, Dropout, Flatten, Embedding, BatchNormalization

In [0]:
df = pd.read_csv('/content/drive/My Drive/Data/fact vs opinion/50k_fo_dataset.csv')

In [0]:
df.head()

,Unnamed: 0,0,1,2,cleaned_corpus,cleaned_corpus_withstops,cleaned_corpus_withstops_withquotes,special_corpus,special_cleaned_corpus2,special_cleaned_corpus5
0,0,In light of some much pain and sorrow from the...,Opinion,Suspecto,In light much pain sorrow country dear communi...,In light of some much pain and sorrow from the...,In light of some much pain and sorrow from the...,In light of some much pain and sorrow from the...,In light of some much pain and sorrow from the...,In light of some much pain and sorrow from the...
1,1,From Conservapedia\n\nSalzburg is the capital ...,Opinion,Suspecto,From Conservapedia Salzburg capital biggest ci...,From Conservapedia\n\nSalzburg is the capital ...,From Conservapedia\n\nSalzburg is the capital ...,From Conservapedia\n\nSalzburg is the capital ...,From Conservapedia\n\nSalzburg is the capital ...,From Conservapedia\n\nSalzburg is the capital ...
2,2,Residents thronged the wide boulevards of Nort...,Factual,newsen,Residents thronged wide boulevards North Korea...,Residents thronged the wide boulevards of Nort...,Residents thronged the wide boulevards of Nort...,Residents thronged the wide boulevards of Nort...,0 0 0 0 0 However journalists’ movements are c...,0 0 0 0 0 However journalists’ movements are c...
3,3,"Spain’s industry minister, Jose Manuel Soria, ...",Factual,newsen,Spains industry minister Jose Manuel Soria Mon...,Spains industry minister Jose Manuel Soria on ...,Spain’s industry minister Jose Manuel Soria on...,Spain’s industry minister Jose Manuel Soria on...,0 0 0 0 0 0 0 0 They have all denied wrongdoing,0 0 0 0 0 0 0 0 They have all denied wrongdoing
4,4,Embattled Daily Show successor Trevor Noah‘s s...,Opinion,Suspecto,Embattled Daily Show successor Trevor Noah ‘ s...,Embattled Daily Show successor Trevor Noah‘s s...,Embattled Daily Show successor Trevor Noah‘s s...,Embattled Daily Show successor Trevor Noah‘s s...,Embattled Daily Show successor Trevor Noah‘s s...,Embattled Daily Show successor Trevor Noah‘s s...


In [0]:
def clean_text_adv(text):
    text = re.sub('\d+\d+', 'NUM', text)
    words = word_tokenize(text)
    clean_words = [re.sub(r'[^A-Za-z]', '', word) for word in words]
    cleaned = ' '.join(clean_words)
    text = cleaned.lower()
    return text

def clean(text, idx):
    try:
        sents = sent_tokenize(text)
        sents1 = [clean_text_adv(i) for i in sents]
        joined = ' '.join(sents1)
        
        print(idx)
    
        return joined
    except:
        sents = text
        print('Ohh exception here')
        sents = clean_text_adv(text)
        b = ' '
        #joined = b.join(sents)
        print(text)
        return sents

In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
corpus = list(df['0'].fillna('0'))
y = [1 if i=='Factual' else 0 for i in list(df['1'])]

In [0]:
corpus[:100]

In [0]:
sample2 = [(a, b) for a,b in zip(corpus, y)]
sample2[0]

('In light of some much pain and sorrow from the country to our dear community. I decided to create this open thread dedicated to gchaucer2 because she was one of my biggest fans and I hers. We connected through this primary and I know we will be celebrating and goofing around in Novemebr.\n\nBuy her book here\n\nAs a gay man , I also will not give in to terror and continue fighting for what I find is the correct path and the right choices to be made in the political arena and beyond.\n\nPolls close in DC at 830EST\n\nFor Orlando\n\nx YouTube Video\n\nFor GC\n\nx YouTube Video\n\nx This is one nation under God, indivisible, with liberty and justice for all. Not just for those who look or love or worship one wayÃ¢Â\x80Â\x94for all. Ã¢Â\x80Â\x94 Hillary Clinton (@HillaryClinton) June 14, 2016',
 0)

In [0]:
sample2[:5]

In [0]:
y = [1 if i=='Factual' else 0 for i in list(df['1'])]

In [0]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
sample = sample2
print(len(sample))

50000


In [0]:
def filterr1(sent):
    a = word_tokenize(sent)
    b = nltk.pos_tag(a)
    c = [i for i in b if i[1] in ['JJ', 'MD']]
    for i in a:
        if i in ['said','told','according','telling','saying','wrote','proposed']:
            return "0"
    if len(c)!=0:
        return sent
    else:
        return "0"
    
def filterr2(sent):
    a = word_tokenize(sent)
    b = nltk.pos_tag(a)
    for i in b:
        if i[1] in ['JJ', 'MD']:
            return "0"   
    return sent
    
def operations1(texts, idx):
    print(idx)
    filter_sample = sent_tokenize(texts) 
    filter_sample2 = [filterr1(i) for i in filter_sample]
    while("0" in filter_sample2):
        filter_sample2.remove("0")
    if len(filter_sample2)!=0:
        filter_sample4 = " ".join(filter_sample2)
        return filter_sample4
    
def operations2(texts, idx):
    print(idx)
    filter_sample = sent_tokenize(texts) 
    filter_sample2 = [filterr2(i) for i in filter_sample]
    while("0" in filter_sample2):
        filter_sample2.remove("0")
    #if "0" in filter_sample2:
     #   filter_sample2.remove("0")
    if len(filter_sample2)!=0:
        filter_sample4 = " ".join(filter_sample2)
        return filter_sample4
#    return filter_sample2

    
updated_samples = [operations1(i[0], idx) if i[1]==0 else operations2(i[0], idx) for idx, i in enumerate(sample)]

In [0]:
updated_samples[:20]

['In light of some much pain and sorrow from the country to our dear community. I decided to create this open thread dedicated to gchaucer2 because she was one of my biggest fans and I hers. We connected through this primary and I know we will be celebrating and goofing around in Novemebr. Buy her book here\n\nAs a gay man , I also will not give in to terror and continue fighting for what I find is the correct path and the right choices to be made in the political arena and beyond. Polls close in DC at 830EST\n\nFor Orlando\n\nx YouTube Video\n\nFor GC\n\nx YouTube Video\n\nx This is one nation under God, indivisible, with liberty and justice for all. Ã¢Â\x80Â\x94 Hillary Clinton (@HillaryClinton) June 14, 2016',
 'From Conservapedia\n\nSalzburg is the capital and biggest city of the federal state Salzburg, Austria.',
 'However, journalists’ movements are closely managed and conversations with the people are monitored. She called the United States an imperialist power that wanted to oc

In [0]:
new_grps = [(i, j) for i,j in zip(updated_samples, y)]

In [0]:
df.to_csv('/content/drive/My Drive/Data/fact vs opinion/FACTVSOP_DATASET.csv', index = False)

# Loading Data and Trainning


In [0]:
df = pd.read_csv('/content/drive/My Drive/Data/fact vs opinion/sent_level/FACTVSOP_DATASET.csv')
df.head(10)

,0,1,cleaned_corpus
0,In light of some much pain and sorrow from the...,0,in light of some much pain and sorrow from the...
1,From Conservapedia\n\nSalzburg is the capital ...,0,from conservapedia salzburg is the capital and...
2,"However, journalists’ movements are closely ma...",1,however journalists movements are closely ma...
3,They have all denied wrongdoing.,1,they have all denied wrongdoing
4,Embattled Daily Show successor Trevor Noah‘s s...,0,embattled daily show successor trevor noah s ...
5,ROCK STAR TREATMENT Musk’s celebrity status pr...,1,rock star treatment musk s celebrity status p...
6,Larry Hunter claims he is a “lifelong conserva...,0,larry hunter claims he is a lifelong conserva...
7,Cyclone Maarutha made landfall on Myanmar's Ra...,1,cyclone maarutha made landfall on myanmar s ra...
8,It is a bit rich for P. Chidambaram to suggest...,0,it is a bit rich for p chidambaram to suggest ...
9,Perpetrators in two of the highest-profile ISI...,0,perpetrators in two of the highestprofile isis...


In [0]:
texts = list(df['0'])
y = list(df['1'])

In [0]:
len(texts)

28230

In [0]:
b = word_tokenize('my name is ashutosh')
len(b)

In [0]:
len(word_tokenize(texts[200]))

In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
cleaned_corpus = [clean(i, idx) for idx, i in enumerate(texts)]
cleaned_corpus

In [0]:
texts = cleaned_corpus
#df['cleaned_corpus'] = cleaned_corpus

In [0]:
max_len = 25
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tk = Tokenizer()
tokenizer = tk.fit_on_texts(pd.Series(texts))
X_tokenized= tk.texts_to_sequences(pd.Series(texts).fillna('UNK'))
X = pad_sequences(X_tokenized, maxlen = max_len, padding = 'post', truncating= 'post')

In [0]:
with open('/content/drive/My Drive/Data/fact vs opinion/TOKENIZER_NEW.h5', 'wb') as handle:
    pickle.dump(tk, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
import pickle
glove = pickle.load(open('/content/drive/My Drive/Glove Emb/news_glove/glove.42B.300d.pkl','rb')) 

In [0]:
a = [(i, j) for i, j in zip(cleaned_corpus, y)]
a[:30]

In [0]:
import numpy as np
EMBEDDING_DIM = 300
word_index = tk.word_index
nb_words = len(word_index) + 1
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in glove: 
        embedding_matrix[i] = glove[word]
    elif word.lower() in glove:
        embedding_matrix[i] = glove[word.lower()]

print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Null word embeddings: 36531


In [0]:
nb_words

122886

In [0]:
z = []
for i in range(embedding_matrix.shape[0]):
    if np.sum(embedding_matrix[i]) == 0:
        z.append(i)

In [0]:
listt = []
for idx, (word, i) in enumerate(word_index.items()):
    if i in z:
        listt.append(word)
    if idx==5000:
        break

In [0]:
embedding_matrix.shape

(122886, 300)

In [0]:
train = np.zeros((X.shape[0], max_len, EMBEDDING_DIM))
for i in range(X.shape[0]):
    train[i] = embedding_matrix[[X[i]]]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
train.shape

(28230, 25, 300)

In [0]:
keras.backend.clear_session()
def lstm_model():
    """
    Model definiton for CPU
    :param nb_words: int-Number of words in embedding layer
    :return: Model
    """
    
    inp = Input(shape=(max_len, 300))
    x = LSTM(15, return_sequences=True)(inp)

    x = Flatten()(x)
    x = Dense(10, activation="relu")(x)
    #x = BatchNormalization()(x)
    #x = Dropout(0.22)(x)
    x = Dense(5, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
model = lstm_model()

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.15, random_state=42)

In [0]:
model = lstm_model()
from keras.callbacks import ModelCheckpoint

filepath="/content/drive/My Drive/Data/fact vs opinion/FACTVSOP_NEW.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]
model = lstm_model()
model.fit(X_train, y_train, batch_size=64, epochs=3, validation_data=(X_test, y_test), verbose=1, callbacks=callbacks)

Train on 23995 samples, validate on 4235 samples
Epoch 1/3
23995/23995 [==============================] - 12s 518us/step - loss: 0.6077 - acc: 0.6711 - val_loss: 0.5826 - val_acc: 0.6848

Epoch 00001: val_loss improved from inf to 0.58256, saving model to /content/drive/My Drive/Data/fact vs opinion/FACTVSOP_NEW.h5
Epoch 2/3
23995/23995 [==============================] - 11s 474us/step - loss: 0.5599 - acc: 0.7000 - val_loss: 0.5676 - val_acc: 0.6911

Epoch 00002: val_loss improved from 0.58256 to 0.56756, saving model to /content/drive/My Drive/Data/fact vs opinion/FACTVSOP_NEW.h5
Epoch 3/3
23995/23995 [==============================] - 11s 473us/step - loss: 0.5418 - acc: 0.7092 - val_loss: 0.5636 - val_acc: 0.6961

Epoch 00003: val_loss improved from 0.56756 to 0.56361, saving model to /content/drive/My Drive/Data/fact vs opinion/FACTVSOP_NEW.h5


In [0]:
model = lstm_model()
model.load_weights('/content/drive/My Drive/Data/fact vs opinion/sent_level/FACTVSOP_NEW.h5')

In [0]:
y_pred = model.predict([X_test], batch_size=1024, verbose=2)
y_pred = (y_pred>0.5) + 0

In [0]:
from sklearn.metrics import classification_report
print(classification_report(y_test ,y_pred))

              precision    recall  f1-score   support

           0       0.85      0.94      0.89      2661
           1       0.87      0.73      0.80      1574

    accuracy                           0.86      4235
   macro avg       0.86      0.83      0.84      4235
weighted avg       0.86      0.86      0.86      4235



In [0]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

[[2492  169]
 [ 424 1150]]


In [0]:
text = ["We should get up early in the morning.", "According to Rahul Gandhi narendra modi is not a good person.", "Taj Mahal is situated in Agra.",
       "I was born on 9th july, 1998 in Kanpur", "Ganga is the longest river in india.", "India won the cricket world cup in 2011.", "Rahul Gandhi said Modi is a good prime minister.",
       "himalyan is situated in the north of india.", "Goa is a good place to be visted. It has so many beautifull beaches.", "She said that This project will be completed in 5 years approx.",
     "This government is not doing work to control the traffic in better manner.", "A nuclear of atom contains proton and neutrons.", "India got freedom in 15th August 1947 and then nehru was the first prime minister of India.",
       "The energy output of the sun is not constant", "Increase of population must be stoped as it is harmfull for various of things.","Delhi is the capital of India",
       "Sun rises in the east ans sets in the west.", "Ohh, How hot this summer is going on", "It looks ligh i will be late in reaching delhi. ",
       "Gandhi Jayanti is celebrated on 2nd October.", "Gandhi ji was born on 2nd october, 1869.", "Gandhi Ji did not do anything for the development of India.",
       "Water flows from high level to low level.", "India might be able to improve its GDP if it invests substantially in major urban infrastructure.",
        "you should get up early in th morning", 'India having 3000 kilometer square area.']

In [0]:
text = ['''Traditional policies offer a wide range of covers within a single product, whereas sachet policies are designed to address specific risks. “Traditional policies are multi-purpose and broader in scope. Sachet policies have targeted coverage duration, and specific sum assured. This makes the underwriting process simpler and the premiums more affordable. Traditional policies generally provide annual, round-the-clock cover with relatively higher sum assured and, therefore, require a more comprehensive underwriting," said Abhishek Bondia, principal officer and managing director, SecureNow.in.

The selling point of sachet policies is the low premium. Toffee Insurance, a digital insurtech startup, offers various bite-sized policies such as fitness insurance, which covers accidental injuries related to a fitness activity or sport, cyclist insurance which covers cycle theft and damage as well as accident-related death or disability, and backpack insurance which covers theft of your backpack or laptop bag. The premiums of these policies range between ₹25 and ₹449 for a time period of 180 days to a year.''']

In [0]:
text = sent_tokenize(text[0])

In [0]:
c = [clean(i, 1) for i in text]

In [0]:
c

['we should get up early in the morning ',
 'according to rahul gandhi narendra modi is not a good person ',
 'taj mahal is situated in agra ',
 'i was born on th july  num in kanpur',
 'ganga is the longest river in india ',
 'india won the cricket world cup in num ',
 'rahul gandhi said modi is a good prime minister ',
 'himalyan is situated in the north of india ',
 'goa is a good place to be visted  it has so many beautifull beaches ',
 'she said that this project will be completed in  years approx ',
 'this government is not doing work to control the traffic in better manner ',
 'a nuclear of atom contains proton and neutrons ',
 'india got freedom in numth august num and then nehru was the first prime minister of india ',
 'the energy output of the sun is not constant',
 'increase of population must be stoped as it is harmfull for various of things ',
 'delhi is the capital of india',
 'sun rises in the east ans sets in the west ',
 'ohh  how hot this summer is going on',
 'it lo

In [0]:
X_tokenized= tk.texts_to_sequences(pd.Series(c).fillna('UNK'))
X = pad_sequences(X_tokenized, maxlen = 25, padding = 'post', truncating= 'post')

In [0]:
x = [embedding_matrix[[i]] for i in X]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


In [0]:
x = np.array(x)

In [0]:
text

In [0]:
y_pred = model.predict(x)
v = [(i, j) for i, j in zip(text, y_pred)]
v

[('We should get up early in the morning.', array([0.4246287], dtype=float32)),
 ('According to Rahul Gandhi narendra modi is not a good person.',
  array([0.175217], dtype=float32)),
 ('Taj Mahal is situated in Agra.', array([0.63421607], dtype=float32)),
 ('I was born on 9th july, 1998 in Kanpur',
  array([0.87088215], dtype=float32)),
 ('Ganga is the longest river in india.', array([0.55003923], dtype=float32)),
 ('India won the cricket world cup in 2011.',
  array([0.8101121], dtype=float32)),
 ('Rahul Gandhi said Modi is a good prime minister.',
  array([0.42209828], dtype=float32)),
 ('himalyan is situated in the north of india.',
  array([0.8190203], dtype=float32)),
 ('Goa is a good place to be visted. It has so many beautifull beaches.',
  array([0.10035551], dtype=float32)),
 ('She said that This project will be completed in 5 years approx.',
  array([0.94266653], dtype=float32)),
 ('This government is not doing work to control the traffic in better manner.',
  array([0.65900